# Sovereign-Doc Cloud Brain\n\nVision-language inference using Qwen2.5-VL-7B on Google Colab T4 GPU.\n\n**Requirements:**\n- GPU Runtime (T4)\n- ngrok Auth Token\n- HuggingFace Token (for model access)

## 1. Install Dependencies

In [ ]:
# Install required packages\n!pip install -q vllm fastapi uvicorn pyngrok python-multipart nest-asyncio loguru pillow

## 2. Configuration

In [ ]:
import os\nfrom google.colab import userdata\n\n# Set authentication tokens\n# Get these from: ngrok.com and huggingface.co\nos.environ['NGROK_AUTH_TOKEN'] = userdata.get('NGROK_TOKEN')\nos.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')\n\nprint('✓ Configuration loaded')

## 3. Upload Code Files\n\nUpload `colab_brain/` directory with:\n- `__init__.py`\n- `inference.py`\n- `server.py`

In [ ]:
# Verify uploaded files\n!ls -la colab_brain/

## 4. Start Server with ngrok Tunnel

In [ ]:
import nest_asyncio\nfrom pyngrok import ngrok\nimport uvicorn\nimport sys\n\n# Enable nested async (required for Colab)\nnest_asyncio.apply()\n\n# Start ngrok tunnel\nngrok.set_auth_token(os.environ['NGROK_AUTH_TOKEN'])\ntunnel = ngrok.connect(8000, bind_tls=True)\n\nprint(f'\\n🚀 Cloud Brain Public URL: {tunnel.public_url}')\nprint(f'\\nEndpoints:')\nprint(f'  - GET  {tunnel.public_url}/health')\nprint(f'  - POST {tunnel.public_url}/analyze')\nprint(f'\\nStarting server...\\n')\n\n# Add colab_brain to Python path\nsys.path.insert(0, '/content')\n\n# Import and run FastAPI server\nfrom colab_brain.server import app\n\n# Run server (blocks until shutdown)\nuvicorn.run(app, host='0.0.0.0', port=8000, log_level='info')

## Testing\n\nTest the endpoints using `curl` or Python requests:\n\n```python\nimport requests\n\n# Health check\nresponse = requests.get(f'{tunnel.public_url}/health')\nprint(response.json())\n\n# Analyze image\nwith open('test_image.jpg', 'rb') as f:\n    files = {'file': f}\n    data = {'query': 'What is in this image?'}\n    response = requests.post(f'{tunnel.public_url}/analyze', files=files, data=data)\n    print(response.json())\n```